In [1]:
# https://python.langchain.com/v0.2/docs/tutorials/sql_qa/

### Security note

General security best practices: https://python.langchain.com/v0.2/docs/security/


### Architecture

1. Convert question to DSL query: Model converts user input to a SQL query.
2. Execute SQL query: Execute the query.
3. Answer the question: Model responds to user input using the query results.

![](https://python.langchain.com/v0.2/assets/images/sql_usecase-d432701261f05ab69b38576093718cf3.png)

- CSV data: https://python.langchain.com/v0.2/docs/how_to/sql_csv/


### Setup DB


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///data/Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### Chains

1. convert the question into a SQL query;
2. execute the query;
3. use the result to answer the original question.


In [3]:
from langchain.chains import create_sql_query_chain

from core.llm import CHAT_LLM


chain = create_sql_query_chain(CHAT_LLM, db)
response = chain.invoke({"question": "How many employees are there"})
response

'SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM "Employee"\nLIMIT 1;'

In [4]:
db.run(response)

'[(8,)]'

- Prompt

  1. Dialect-specific. In this case it references SQLite explicitly.
  2. Has definitions for all the available tables.
  3. Has three examples rows for each table.

- Official reference: https://arxiv.org/pdf/2204.00498.pdf


In [6]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [5]:
# create_sql_query_chain()
_sqlite_prompt = """You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

"""
PROMPT_SUFFIX = """Only use the following tables:
{table_info}

Question: {input}"""

### Execute SQL query

This is the most dangerous part of creating a SQL chain. \
Minimize the database connection permissions as much as possible.


In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(CHAT_LLM, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

### Answer the question


In [10]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | CHAT_LLM
    | StrOutputParser()
)
chain.invoke({"question": "How many employees are there"})

'There are a total of 8 employees.'

Let's review what is happening in the above LCEL. Suppose this chain is invoked.

1. After the first `RunnablePassthrough.assign`, we have a runnable with two elements: \
   `{"question": question, "query": write_query.invoke(question)}` \
   Where `write_query` will generate a SQL query in service of answering the question.
2. After the second `RunnablePassthrough.assign`, we have add a third element "result" that contains execute_query.invoke(query), where query was computed in the previous step.
3. These three inputs are formatted into the prompt and passed into the LLM.
4. The `StrOutputParser()` plucks out the string content of the output message.


### Next steps

- [Prompting strategies](https://python.langchain.com/v0.2/docs/how_to/sql_prompting/): Advanced prompt engineering techniques.
- [Query checking](https://python.langchain.com/v0.2/docs/how_to/sql_query_checking/): Add query validation and error handling.
- [Large databases](https://python.langchain.com/v0.2/docs/how_to/sql_large_db/): Techniques for working with large databases.


### Agents

The main advantages of using the SQL Agent are:

- It can answer questions based on the databases' schema as well as on the databases' content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can query the database as many times as needed to answer the user question.
- It will save tokens by only retrieving the schema from relevant tables.


In [11]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=CHAT_LLM)
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f61f28188d0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f61f28188d0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f61f28188d0>),
 QuerySQLCheckerTool(description='Use this tool to double check

#### System prompt


In [12]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

#### Initializing agent


In [13]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(CHAT_LLM, tools, messages_modifier=system_message)

for s in agent_executor.stream({"messages": [HumanMessage(content="Which country's customers spent the most?")]}):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sC7WYfHNhuDQHDL0DGa95zyD', 'function': {'arguments': '{"query":"SELECT c.Country, SUM(i.Total) AS Total_Spent FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId GROUP BY c.Country ORDER BY Total_Spent DESC LIMIT 1"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 557, 'total_tokens': 610}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-82655551-7486-45e1-ba94-bf90af5dbe52-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': 'SELECT c.Country, SUM(i.Total) AS Total_Spent FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId GROUP BY c.Country ORDER BY Total_Spent DESC LIMIT 1'}, 'id': 'call_sC7WYfHNhuDQHDL0DGa95zyD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 557, 'outp

Note that the agent executes multiple queries until it has the information it needs:

1. List available tables;
2. Retrieves the schema for three tables;
3. Queries multiple of the tables via a join operation.


In [14]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Describe the playlisttrack table")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7kkAKIYkYMFWRFMrodXl8GJG', 'function': {'arguments': '{"table_names":"playlisttrack"}', 'name': 'sql_db_schema'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 554, 'total_tokens': 571}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-78358459-e55d-4f16-82cc-a32a67be6218-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'playlisttrack'}, 'id': 'call_7kkAKIYkYMFWRFMrodXl8GJG', 'type': 'tool_call'}], usage_metadata={'input_tokens': 554, 'output_tokens': 17, 'total_tokens': 571})]}}
----
{'tools': {'messages': [ToolMessage(content="Error: table_names {'playlisttrack'} not found in database", name='sql_db_schema', tool_call_id='call_7kkAKIYkYMFWRFMrodXl8GJG')]}}
----
{'agent': {'messages': [AIMessage(content='I apologize f

### Dealing with high-cardinality columns


In [17]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")
albums[:5]

['Pop',
 'The Best Of Men At Work',
 'The Office, Season',
 'Are You Experienced?',
 'The Best of Beethoven']

Using this function, we can create a retriever tool that the agent can execute at its discretion.


In [20]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from core.llm import EMBEDDINGS


vector_db = FAISS.from_texts(artists + albums, EMBEDDINGS)
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [21]:
print(retriever_tool.invoke("Alice Chains"))

Alice In Chains

Whitesnake

Pearl Jam

Pearl Jam

Alanis Morissette


- Putting this together:


In [23]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)
tools.append(retriever_tool)
agent = create_react_agent(CHAT_LLM, tools, messages_modifier=system_message)

In [26]:
for s in agent.stream(
    {"messages": [HumanMessage(content="How many albums does alis in chain have?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wPrw9BaTaLo3GjfpszPw69ns', 'function': {'arguments': '{"query":"alis in chain"}', 'name': 'search_proper_nouns'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 736, 'total_tokens': 755}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-5db7d608-768f-4668-9b25-014fa9c88cb0-0', tool_calls=[{'name': 'search_proper_nouns', 'args': {'query': 'alis in chain'}, 'id': 'call_wPrw9BaTaLo3GjfpszPw69ns', 'type': 'tool_call'}], usage_metadata={'input_tokens': 736, 'output_tokens': 19, 'total_tokens': 755})]}}
----
{'tools': {'messages': [ToolMessage(content='Alice In Chains\n\nIn Step\n\nIV\n\nTransmission\n\nCoda', name='search_proper_nouns', tool_call_id='call_wPrw9BaTaLo3GjfpszPw69ns')]}}
----
{'agent': {'messages': [AIMessage(content='', additi